# Draft Prep

Lots prepare csv files for draft activity. Also prepare weighted csv files for other uses.

In [1]:
import os
import sys
import warnings
import re

import pandas as pd

sys.path.append(r'/home/lenhart/Repos/phi-utils')

from philosofool.data_sources.utils import read_yml
from fantasy_baseball_draft.utils import StatSynonyms, load_cbs_data, DataLoader
from fantasy_baseball_draft.utils import cbs_player_col_to_df


data_path = read_yml('local/config.yml')['paths']['local_data']


## ID Functions

In [2]:
class AssociatePlayers:
    """Associate entities from different datasets.

    Given two dataframes that share entities but not a common single
    join key, this can construct a merge of the data based on columns
    that are assumed to find unique one.

    Example: Given dataframes of pitchers in 2019 that use different
    naming conventions ('Mike', 'Michael', etc.) we can align these by
    assuming that two pitchers are the same if they started the same
    number of games, had the same number of walks, hits and strikeouts.
    """
    synonyms = StatSynonyms()

    def associate(self, df1: pd.DataFrame, df2: pd.DataFrame, index_cols: list) -> pd.DataFrame:
        if df1.duplicated().sum():
            warnings.warn("Found duplicated in df1.")
        if df2.duplicated().sum():
            warnings.warn("Found duplicated in df2.")
        df1 = self.synonyms.normalize_df(df1)
        df2 = self.synonyms.normalize_df(df2)
        df = df1.set_index(index_cols).merge(df2.set_index(index_cols), left_index=True, right_index=True)
        return df


In [ ]:
def merge_on_name(cbs: pd.DataFrame, fg: pd.DataFrame) -> pd.DataFrame:
    cbs_player = (
        cbs_player_col_to_df(cbs.Player) #.Team.fillna('---').unique().tolist()
        .merge(cbs[['Player']], left_index=True, right_index=True)
        .merge(fg[['Name', 'playerid']], on='Name', how='left')
    )
    return cbs_player

def map_cbs_player_col_to_id_by_name(cbs: pd.DataFrame, fg: pd.DataFrame) -> pd.Series:
    """Use cbs "Player" field to return series of CBS player names to fg player ids."""
    df = merge_on_name(cbs, fg).drop_duplicates(subset=['playerid'], keep=False)
    playerid = df.playerid
    ids = df.set_index('Player').playerid.dropna().astype(str)
    return ids.where(ids.str.startswith('sa'), ids.str.strip('sa').astype(int))


In [ ]:
def build_id_map(df: pd.DataFrame, fg_df: pd.DataFrame, ids: pd.Series) -> pd.Series:
    """Map cbs Player column to fg ids.

    Parameters
    ----------
    df:
        CBS data.
    fg_df:
        Fangraphs data.
    ids:
        Mapping of Player:fg_id; this is a mapping of known cases.
    """
    df = df.copy()
    df['playerid'] = df.Player.map(ids).fillna(-1)
    name_ids = map_cbs_player_col_to_id_by_name(df[df.playerid == -1], fg_df)
    name_ids = name_ids[~name_ids.duplicated()]
    ids = pd.concat([ids, name_ids]).to_dict()
    return df.Player.map(ids).fillna(-1)

In [ ]:
def _build_id_map_from_stat_associations(cbs, fg, index_cols, duplicated=False) -> pd.Series:
    """Return series mapping CBS player "Player" column to a fangraphs ID column."""
    cbs_with_playtime = cbs[cbs[index_cols].sum(axis=1) > 0]
    n_players_with_pt = len(cbs_with_playtime)
    if duplicated:
        return cbs_with_playtime[cbs_with_playtime.duplicated(subset=index_cols, keep=False)]
    cbs_with_playtime = cbs_with_playtime.drop_duplicates(subset=index_cols, keep=False)
    print(f"Dropped {n_players_with_pt - len(cbs_with_playtime)} duplicated records.")

    cbs_to_fg = AssociatePlayers().associate(cbs_with_playtime, fg, index_cols)
    cbs_to_fg.Player = cbs_to_fg.Player.str.strip()
    as_dict = dict(zip(cbs_to_fg.Player.str.strip(), cbs_to_fg.playerid))
    return pd.Series(as_dict)


## Add Fangraphs Player Id to CBS data

In [ ]:
hitter_match = ['AB', 'H', 'BB', 'RBI', 'K']
pitcher_match = ['IP', 'W', 'G', 'K', 'H', 'W']

In [ ]:
data_path = read_yml('local/config.yml')['paths']['local_data']
hist_path = os.path.join(data_path, 'historical')
loader = DataLoader(hist_path)

pitcher_ids = _build_id_map_from_stat_associations(
    loader.load_cbs_csv('cbs_pitchers_2023.csv'),
    loader.load_csv('fg_pitchers_2023.csv'), pitcher_match
)

hitter_ids = _build_id_map_from_stat_associations(
    loader.load_cbs_csv('cbs_hitters_2023.csv'),
    loader.load_csv('fg_hitters_2023.csv'), hitter_match
)
#build_id_map('cbs_hitters_2022.csv', 'fg_hitters_2022.csv', ['H', 'BB', 'RBI', 'K'], True)
#{k: v for k, v in hitter_ids.items() if 'Franc' in k}# [hitter_ids..str.contains('Franc')]
hitter_ids

In [ ]:
projections = DataLoader(os.path.join(data_path, 'projections/2024'))
fg_hitter_proj = projections.load_csv('fg_depth_hitters.csv')
fg_pitcher_proj = projections.load_csv('fg_depth_pitchers.csv')
cbs_hitter_proj = (
    projections
    .load_cbs_csv('cbs_hitters.csv')
    .assign(
        playerid=lambda df: build_id_map(df, fg_hitter_proj, hitter_ids),
        PA=lambda df: df.AB + df.BB
    )
)

cbs_pitchers_proj = (
    projections
    .load_cbs_csv('cbs_pitchers.csv')
    .assign(
        playerid=lambda df: build_id_map(df, fg_pitcher_proj, pitcher_ids),
        ER=lambda df: df.ERA * df.IP / 9
    )
)


In [ ]:
fg_hitter_proj.head().columns

In [ ]:
cbs_pitchers_proj.head(3)


In [ ]:
cbs_hitter_proj.head(3)

## Combine Projections

In [18]:
def merge_fg(cbs, base_path, cols, name_filter=lambda x: x):
    cbs = cbs.copy()
    ids = cbs.playerid.to_numpy()
    loader = DataLoader(base_path)
    for path in os.listdir(base_path):
        if name_filter(path) and 'cbs' not in path:
            #print(path)
            suffixes = ('', '_' + path[:5].strip('_'))
            df = loader.load_csv(path)
            if len(df) == 0:
                #print(f'{path} data is empty...')
                continue
            #print(df.columns, df.playerid[:10])
            df = df[~df.playerid.astype(str).str.startswith('sa')]

            df.playerid = df.playerid.astype(int)
            df = df[df.playerid.isin(list(ids))]
            assert len(df) > 0, 'df empty, which is sort of like wdf?'
            cbs = cbs.merge(df[cols], on='playerid', suffixes=suffixes, how='left')
    return cbs

def merge_fg_hitters(cbs, base_path):
    cols = ['playerid', 'PA',  'AB', 'H', 'HR', 'R', 'RBI', 'SB']
    return merge_fg(cbs, base_path, cols, lambda x: 'hitter' in x)

def merge_fg_pitchers(cbs, base_path):
    cols = ['playerid', 'IP', 'ER', 'WHIP', 'K', 'S', 'W']
    return merge_fg(cbs, base_path, cols, lambda x: 'pitch' in x)


In [19]:
def add_weighted_stats(df, stats, playtime_stat):
    df = df.copy()
    pt = playtime_stat
    suffixes = set(['_'.join(col.split('_')[1:]) for col in df.columns if len(col.split('_')) > 1])
    assert (f'{pt}_fg-de' in df.columns), f"{pt} not in fg-de columns. Something is wrong."
    df[f'{pt}_adj'] = (df[f'{pt}_fg-de'] + df[pt]) / 2

    for suf in suffixes:
        for col in stats:
            df[f'{col}_{suf}'] = df[f'{col}_{suf}'] / df[f'{pt}_{suf}'] * df[f'{pt}_adj']
    for col in stats:
        adj_cols = [col] + [c for c in df.columns if re.match(f'{col}_', c)]
        df[f'{col}_adj'] = (df[adj_cols].sum(axis=1) / df[adj_cols].notna().sum(axis=1))
    return df

#add_weighted_stats(cbs_hitters, cols[2:], 'PA')
hitter_stats = ['playerid', 'PA',  'AB', 'H', 'HR', 'R', 'RBI', 'SB']
pitcher_stats = ['playerid', 'IP', 'ER', 'WHIP', 'K', 'S', 'W']
full_hitters = (cbs_hitters
    .pipe(merge_fg_hitters, data_path + '/projections')
    .pipe(add_weighted_stats, hitter_stats[2:], 'PA')
    )
full_pitchers = (cbs_pitchers
    .pipe(merge_fg_pitchers, data_path + '/projections')
    .pipe(add_weighted_stats, pitcher_stats[2:], 'IP')
)

In [20]:
from fantasy_baseball_draft.spg import FantasyValuator, spgs_from_standings_html

valuator = FantasyValuator(spgs_from_standings_html(os.path.join(data_path, 'standings/cbs_2021_standings.html')))


In [21]:
elig = DataLoader(os.path.join(data_path, 'eligibility')).load_cbs_csv('eligibility.csv')

def extract_projection(df):
    extract = ['Avail', 'Player', 'playerid'] + [col for col in df.columns if '_adj' in col]
    df = df[extract].rename(columns={k: re.sub('_adj', '', k) for k in extract})
    return df

hitter_proj = extract_projection(full_hitters).fillna({'Eligible': 'U'})
print(hitter_proj.PA.isna().sum())
hitter_proj = hitter_proj.merge(elig[['Player', 'Eligible']],  how='left', on='Player').fillna({'Eligible': 'U'})
hitter_proj.isna().sum()
#hitter_proj[hitter_proj.Eligible.isna()]


3290


Avail          0
Player         0
playerid       0
PA          3296
AB             0
H              0
HR             0
R              0
RBI            0
SB             0
Eligible       0
dtype: int64

In [22]:
hitter_proj['fwar'] = valuator.valuate_hitters(hitter_proj, 16*12)

pitcher_proj = extract_projection(full_pitchers)
pitcher_proj['ERA'] = pitcher_proj.ER / pitcher_proj.IP * 9
pitcher_proj['fwar'] = valuator.valuate_pitchers(pitcher_proj, 16*12)

hitter_proj.sort_values('fwar', ascending=False).head(20)

,Avail,Player,playerid,PA,AB,H,HR,R,RBI,SB,Eligible,fwar
0,You Drink Bitches' Wine,Aaron Judge CF | NYY,15640,651.5,549.227049,155.732473,44.758282,108.963063,111.441546,10.924183,"CF,RF,U",11.364055
11,Walk-In Clostes,Fernando Tatis SS | SD,19709,561.0,497.337000,140.340993,38.455871,98.935874,96.508660,24.612627,"SS,U",11.307351
2,Walk-In Clostes,Trea Turner SS | PHI,16252,654.0,598.276754,177.472117,21.620289,95.898040,80.033646,28.285110,"SS,U",10.342746
4,Walk-In Clostes,Ronald Acuna RF | ATL,18401,622.5,535.416395,146.056880,28.094484,100.583261,77.202600,33.029117,"RF,U",10.188892
1,Walk-In Clostes,Jose Ramirez 3B | CLE,13510,651.0,571.360440,156.525145,30.826891,93.879379,105.123594,22.397414,"3B,U",10.131490
3,Walk-In Clostes,Julio Rodriguez CF | SEA,23697,641.5,580.308216,159.926005,29.373088,94.208325,88.046073,25.939936,"CF,U",9.744740
10,The Tucker and the Hare,Freddie Freeman 1B | LAD,5361,664.0,573.815127,173.473741,25.634954,102.563525,95.237498,9.736378,"1B,U",9.359388
6,The Tucker and the Hare,Kyle Tucker RF | HOU,18345,620.5,547.865760,150.416512,30.602224,86.213195,99.921124,21.325625,"RF,U",9.299544
13,Omak Wrong Players,Vladimir Guerrero 1B | TOR,19611,639.5,566.849436,164.487096,35.296484,95.066169,103.336167,6.314437,"1B,U",9.092797
9,Omak Wrong Players,Juan Soto RF | SD,20123,679.5,533.764584,149.594083,32.475459,111.996352,90.957444,9.405037,"RF,U",8.814019


In [23]:
hitter_proj.to_csv('local/hitter_proj.csv', index_label='index')
pitcher_proj.to_csv('local/pitcher_proj.csv', index_label='index')

## Matching Unfound Players

Everything above is solid. WIP stuff to find more player ids.

In [ ]:
france = unfound_hitters.loc[93, 'Player'] # in hitter_ids
cbs_player_col_to_df(unfound_hitters.Player)#.merge()
cbseam = cbs_player_col_to_df(cbs22.Player).Team.fillna('---').unique().tolist()
teams = list(zip(
    sorted(cbseam),
    sorted(fg22.Team.unique().tolist())
))
{b: a for a, b in teams if a != b}

{'- - -': '---',
 'KCR': 'KC',
 'SDP': 'SD',
 'SFG': 'SF',
 'TBR': 'TB',
 'WSN': 'WAS'}